In [1]:
import os
import requests
import logging
from docling.document_converter import DocumentConverter
from langchain_ibm import WatsonxLLM, ChatWatsonx
from langchain_core.prompts import PromptTemplate
from ibm_granite_community.notebook_utils import get_env_var
from transformers import AutoTokenizer

import pandas as pd
import sqlite3
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough

from dotenv import load_dotenv

load_dotenv()

c:\Users\micae\OneDrive\Área de Trabalho\Projetos\IBM Hackathon\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

### Loading the model

In [35]:
ibm_cloud_api_key = get_env_var('WATSONX_APIKEY')
project_id = get_env_var('WATSONX_PROJECT_ID')
watson_url = get_env_var('WATSONX_URL')

watson_llm = WatsonxLLM(
    model_id="ibm/granite-3-8b-instruct",
    apikey=ibm_cloud_api_key,
    project_id=project_id,
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 2000,
        "min_new_tokens": 1,
        "repetition_penalty": 1,
        "temperature":0.8,
        "stop_sequences" : ["END","Observation:","PAUSE"]
    },
    url=watson_url,
)

### Defining tools

In [27]:
from typing import Optional, Type, List
from pydantic import  Field
from pydantic import BaseModel
from langchain_core.tools import BaseTool
from langchain.agents import tool
import streamlit as st


class Params(BaseModel):
    regulatory_summary: dict = Field(description="Regulatory summaty ")
    impact_analysis: dict = Field(description="Impact Analysis")
    action_plan: dict = Field(description="Action plan ")
    final_recommendations: dict = Field(description="final recommendations ")

@tool(args_schema=Params)
def show_analisys_to_user(**document):
    """Use this action to show to the user the Regulatory Impact Analysis Document
    """
    print("Mostrando DOC-----------\n",document)
    return "The document was shown with success."


tools=[show_analisys_to_user]

In [28]:
show_analisys_to_user.args

{'regulatory_summary': {'description': 'Regulatory summaty ',
  'title': 'Regulatory Summary',
  'type': 'object'},
 'impact_analysis': {'description': 'Impact Analysis',
  'title': 'Impact Analysis',
  'type': 'object'},
 'action_plan': {'description': 'Action plan ',
  'title': 'Action Plan',
  'type': 'object'},
 'final_recommendations': {'description': 'final recommendations ',
  'title': 'Final Recommendations',
  'type': 'object'}}

### Defining Prompt

In [29]:
system_prompt2 = """
You are an advanced conversational AI assistant specialized in regulatory impact analysis for financial institutions. Your goal is to analyze new regulations, assess their impact, and generate structured compliance reports.  

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer. 
You ALWAYS respond in a JSON blob.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then PAUSE and wait o be called again with observation.
Observation will be the result of running those actionsand will be return to you.

Your available actions are:
{tools}

Valid "action" values are: "Final Answer" or {tool_names}. Use "action" value "Final Answer" to talk to the user.
Provide only ONE action per $JSON_BLOB, as shown:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```
END

You follow a dynamic and iterative process:
- Generate an initial regulatory impact analysis and show to use by the action "show_analisys_to_user".  
- Allow the user to refine or modify the analysis.
- Update and adjust documents in real-time based on user feedback.  
- Keep track of previous interactions to ensure consistency.  

STRICT RULES:
- The only Valid "action" values are: "Final Answer" or {tool_names}.
- You must NEVER pass the document by the action "Final Answer".
- After the document be shown to the user with succsess, tell tah user "The document was shown with success".
- Your final output should always be directly based on the Observation result.
- The document [Document] must always be passed as a dict objetc to "action_input".
- The "action_input" for the action "Final Answer" must be always a string.
- Always put a "END" and end of $JSON_BLOB.
- IMPORTANT: ALWAYS put the $JSON_BLOB between triple backticks.
- Always respond with a  $JSON_BLOB with format:
```
{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}
```

END

---

### Step-by-Step Interaction Flow  

1. Understanding the Regulation (Initial Analysis)  
- Read and summarize the regulatory document.  
- Extract key requirements and deadlines.  
- Identify the business areas impacted.  

2. Impact Analysis & Risk Assessment 
- Assess operational, financial, and legal risks.  
- Define process changes and RACI responsibilities.  
- Suggest mitigation strategies.  

3. Show document created to the user, using the action "show_analisys_to_user".

4. Interactive Document Editing (User Feedback & Adjustments)  
- The user can request modifications to any section.  
- You ask clarifying questions if needed.  
- All changes should be reflected dynamically in the generated documents, end exibiting with the action "show_analisys_to_user".  

5. Action Plan & Report Generation
- Summarize the final analysis in a structured report.  
- Provide recommendations and next steps.  
- Ensure the final document aligns with compliance requirements.  

---

## Example Conversation Flow  

User: "Generate a compliance report with these changes."  
AI: "Here's the updated compliance report. Let me know if you'd like any further modifications."   
Action: The final document is generated based on the user's inputs.
```
{{
  "action": show_analisys_to_user,
  "action_input": [Document].
}}
```
END

Observation: The document was shown with success.
Thought: The user has confirmed that the document was shown with success.

Action: Final Answer
```
{{
  "action": "Final Answer",
  "action_input": "Tell me if you need more help."
}}
```
END

---

## Document Format Example 

**Document**
Regulatory Summary 
- Regulation Title: [Name]  
- Key Requirements: [Summary]  
- Deadline: [Date]  

Impact Analysis
- Impacted Areas: Compliance, IT, Risk, **Operations**  
- Required Changes: Update AML monitoring system, revise policies, retrain staff  

Action Plan
- Task: Update reporting framework  
- Owner: Compliance Team  
- Deadline: 30 days  
- Status: Pending  

Final Recommendations  
- [List of actions required for full compliance]  

END

## Analise the following banking regulation:

{regulation}

"""

In [30]:
regulation = """12 U.S. Code § 4301 - Findings and purpose
(a)Findings
The Congress hereby finds that economic stability would be enhanced, competition between depository institutions would be improved, and the ability of the consumer to make informed decisions regarding deposit accounts, and to verify accounts, would be strengthened if there was uniformity in the disclosure of terms and conditions on which interest is paid and fees are assessed in connection with such accounts.

(b)Purpose
It is the purpose of this chapter to require the clear and uniform disclosure of—
(1)the rates of interest which are payable on deposit accounts by depository institutions; and
(2)the fees that are assessable against deposit accounts,
so that consumers can make a meaningful comparison between the competing claims of depository institutions with regard to deposit accounts.
"""

In [31]:
human_prompt = """<|start_of_role|>user<|end_of_role|>{input}<|end_of_text|>"""
assistant = """<|start_of_role|>assistant<|end_of_role|>{agent_scratchpad}<|end_of_text|>"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt2),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", human_prompt),
        ("assistant",assistant)
    ]
)

prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
    regulation=regulation,
)

In [21]:
prompt

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

### Defining chain

In [37]:
memory = ConversationBufferMemory()

chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | watson_llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory, max_iterations= 3
)

In [36]:
memory.clear()

In [38]:
result = agent_executor.invoke({"input":""})



> Entering new AgentExecutor chain...

{
  "action": "show_analisys_to_user",
  "action_input": {
    "regulatory_summary": {
      "description": "The Congress finds that uniformity in disclosure of interest rates and fees on deposit accounts enhances economic stability, improves competition, and strengthens consumer decision-making.",
      "title": "Regulatory Summary",
      "type": "object"
    },
    "impact_analysis": {
      "description": "The regulation impacts Compliance, IT, and Operations departments. Required changes include updating disclosure policies, systems, and staff training.",
      "title": "Impact Analysis",
      "type": "object"
    },
    "action_plan": {
      "description": "Task: Update disclosure policies and systems. Owner: Compliance Team. Deadline: 6 months. Status: Pending.",
      "title": "Action Plan",
      "type": "object"
    },
    "final_recommendations": {
      "description": "1. Review and update disclosure policies. 2. Implement new syst

In [39]:
result

{'input': '',
 'history': '',
 'output': 'Do you need further assistance with this regulation?'}

In [40]:
result = agent_executor.invoke({"input":"I need more recomendation. It's needed to envolve Operations?"})



> Entering new AgentExecutor chain...
{
  "action": "show_analisys_to_user",
  "action_input": {
    "regulatory_summary": {
      "description": "The Congress finds that uniformity in disclosure of interest rates and fees on deposit accounts enhances economic stability, improves competition, and strengthens consumer decision-making.",
      "title": "Regulatory Summary",
      "type": "object"
    },
    "impact_analysis": {
      "description": "The regulation impacts Compliance, IT, and Risk departments. Operations may also be involved in updating disclosure practices and training staff.",
      "title": "Impact Analysis",
      "type": "object"
    },
    "action_plan": {
      "description": "Task: Update disclosure practices and staff training. Owner: Compliance Team. Deadline: 90 days. Status: Pending.",
      "title": "Action Plan",
      "type": "object"
    },
    "final_recommendations": {
      "description": "1. Update disclosure practices to ensure uniformity in interes

In [41]:
result

{'input': "I need more recomendation. It's needed to envolve Operations?",
 'history': 'Human: \nAI: Do you need further assistance with this regulation?',
 'output': 'The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements.'}

In [42]:
result = agent_executor.invoke({"input":"OK. Thanks for the analisys!"})



> Entering new AgentExecutor chain...
```
{
  "action": "show_analisys_to_user",
  "action_input": {
    "regulatory_summary": {
      "description": "The Congress finds that uniformity in disclosure of interest rates and fees on deposit accounts enhances economic stability, improves competition, and strengthens consumer decision-making.",
      "title": "Regulatory Summary",
      "type": "object"
    },
    "impact_analysis": {
      "description": "The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements.",
      "title": "Impact Analysis",
      "type": "object"
    },
    "action_plan": {
      "description": "No specific tasks or deadlines are mentioned in the regulation. However, the Operations department should be prepared to implement changes and monitor compliance.",
      "title": "Action Plan",
      "type": "object"
    },
    "final_recommendations": {
      "description": "The Operations departm

In [43]:
result

{'input': 'OK. Thanks for the analisys!',
 'history': "Human: \nAI: Do you need further assistance with this regulation?\nHuman: I need more recomendation. It's needed to envolve Operations?\nAI: The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements.",
 'output': 'Is there anything else you would like to know about this regulation?'}

In [46]:
result = agent_executor.invoke({"input":"No, thanks."})



> Entering new AgentExecutor chain...
{
  "action": "Final Answer",
  "action_input": "The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements. No further assistance is needed."
}

END

> Finished chain.


In [47]:
result

{'input': 'No, thanks.',
 'history': "Human: \nAI: Do you need further assistance with this regulation?\nHuman: I need more recomendation. It's needed to envolve Operations?\nAI: The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements.\nHuman: OK. Thanks for the analisys!\nAI: Is there anything else you would like to know about this regulation?\nHuman: No, thanks.\nAI: The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements. Is there anything else you would like to know about this regulation?",
 'output': 'The Operations department should be involved in implementing changes and monitoring compliance with the new disclosure requirements. No further assistance is needed.'}